In [9]:
import os, re, time, json
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np
import tensorflow as tf
import google
from matplotlib import pyplot as plt
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.3.0


In [2]:
from src.generators.tf_records_generator import get_dataset
from src.models.efficientNet_B0 import EffNet0
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

In [10]:
CONFIG = dict(
    replicas=1,
    input_shape=(256, 256, 3),
    image_resolution=256,
    batch_size=8,
    epochs=5,
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [12]:
IS_COLAB_BACKEND = 'COLAB_GPU' in os.environ  # this is always set on Colab, the value is 0 or 1 depending on GPU presence
if IS_COLAB_BACKEND:
  from google.colab import auth
  # Authenticates the Colab machine and also the TPU using your
  # credentials so that they can access your private GCS buckets.
  auth.authenticate_user()
  data_dir = 'gs://dataset_files/'
else:
  data_dir = 'dataset/'

In [5]:
# Detect hardware
try:
  tpu_resolver = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu_resolver = None
  gpus = tf.config.experimental.list_logical_devices("GPU")

# Select appropriate distribution strategy
if tpu_resolver:
  tf.config.experimental_connect_to_cluster(tpu_resolver)
  tf.tpu.experimental.initialize_tpu_system(tpu_resolver)
  strategy = tf.distribute.TPUStrategy(tpu_resolver)
  print('Running on TPU ', tpu_resolver.cluster_spec().as_dict()['worker'])
elif len(gpus) > 1:
  strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
  print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on single GPU ', gpus[0].name)
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on CPU')
  
print("Number of accelerators: ", strategy.num_replicas_in_sync)
CONFIG["replicas"] = strategy.num_replicas_in_sync


Running on single GPU  /device:GPU:0
Number of accelerators:  1


In [6]:
path = data_dir + str(CONFIG["image_resolution"])
files_train = np.sort(np.array(tf.io.gfile.glob(path + '/train*.tfrec')))
ds = get_dataset(files_train, CONFIG)


Tensor("ParseSingleExample/ParseExample/ParseExampleV2:3", shape=(), dtype=string)


In [7]:
with strategy.scope():
  model = EffNet0(config=CONFIG, trainable_base=True)
  model.compile(optimizer=CONFIG["optimizer"], loss=CONFIG["loss"], metrics=['accuracy', keras.metrics.AUC()])



In [8]:
history = model.fit(ds, batch_size=CONFIG["batch_size"]*CONFIG["replicas"], epochs=CONFIG["epochs"])

Epoch 1/5
    185/Unknown - 40s 217ms/step - loss: 0.2613 - accuracy: 0.9716 - auc: 0.5917   

KeyboardInterrupt: 